# LEGOとAIカメラの接続
下記の写真のように、LEGOとAIカメラを接続してください。LEGOのポートは [F] を使ってください。

# PythonによるAIカメラの操作

まずはPythonによりAIカメラの顔認識を行います。LEGO SPIKEから以下のプログラムを打ち込んでください。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

上記プログラムはLEGO SPIKEに接続したAIカメラを使うための準備です。非常にややこしいのですが、今時点ではこういうものと思ってください。

- from huskylens_lib import HuskyLensという行がありますが、これはHuskyLensを使うための関数を取り込む作業をしています。
- port.F.mode(,...)という行は、LEGO SPIKE HUBの　FポートとAIカメラを接続するように指示しています。
- husky = HuskyLens(port.F) では AIカメラ(HuskyLens）を使えるように準備しています。

上記プログラムコードによりAIカメラを使う準備ができました。手でカメラを操作していた時は、ダイアルを動かして顔認識モードに切り替えをしていました。この作業はプログラムで行えます。以下のコードを最後の行に追加してください。

In [ ]:
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)

上記関数を実行すると、AIカメラが顔認識モードに切り替わります。

以上の作業によりAIカメラが顔認識モードで動作可能になりました。AIカメラとLEGOを組み合わせるには、AIカメラに対して、今誰かカメラに写っているか？問い合わせる必要があります。問い合わせるための関数も用意されています。

In [ ]:
husky.read_blocks()

上記関数を実行すると、カメラに学習ずみの顔が映っているかどうかを返却します。もし学習済みの顔が映っていると以下と返却します。

In [ ]:
>>> husky.read_blocks()
[('block', 1, 164, 141, 109, 146)]

もしだれの顔も映っていない場合は以下と返却します。空のリストを返却します。

In [ ]:
>>> husky.read_blocks()
[]

もし顔が映っているが知らない人の場合、リストの中のリストの２番目の値が0となります。これはID:0であることを示しています。

In [ ]:
[('block', 0, 137, 140, 48, 82)]

では、一度だけではなくて何度も繰り返し確認するプログラムに修正しましょう。プログラム全文は以下となります。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)        # AIカメラ初期化
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)  # 顔認識モードに設定

while True:
    print(husky.read_blocks())   # 認識状態を取得、表示

上記プログラムを実行して画面に表示される文字列を確認してください。
学習した人の顔を映した時、学習していない顔を映した時で表示が変わるのを確認してください

# 知っている顔を映したら音を流しましょう

覚えている人の顔がカメラに映ったら　関数 husky.read_blocks()　が返却するリストの中のIDが１　になります。if  文で、IDが　１かどうかを判断して、1 だったら音を鳴らします。  